In [1]:
import requests
import json
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 


In [530]:
match_master_list=list()
#La liga PD
#BPL PL
#Bundes BL1
#Liga portuguesa PPL
#França FL1
#Italia SA
for year in range(2017,2020):   
    uri = 'http://api.football-data.org/v2/competitions/PD/matches?season=' + str(year)
    headers = { 'X-Auth-Token': '291d3414a2f641ba9d97102df496e9cd' }
    response = requests.get(uri, headers=headers)
        
    match_list=response.json()['matches']
    #match_list=match_list[::-1]
    match_master_list.append(match_list)
    nr_games=len(match_list)
    teams_list=list()
    for i in range (0, nr_games):
        teams_list.append(match_list[i]['homeTeam']['name'])
    if year == 2019:
        teams_list=np.unique(teams_list)
        print("List of teams: " ,teams_list)
        team=input("Team name:")

List of teams:  ['Athletic Club' 'CA Osasuna' 'CD Leganés' 'Club Atlético de Madrid'
 'Deportivo Alavés' 'FC Barcelona' 'Getafe CF' 'Granada CF' 'Levante UD'
 'RC Celta de Vigo' 'RCD Espanyol de Barcelona' 'RCD Mallorca'
 'Real Betis Balompié' 'Real Madrid CF' 'Real Sociedad de Fútbol'
 'Real Valladolid CF' 'SD Eibar' 'Sevilla FC' 'Valencia CF'
 'Villarreal CF']
Team name:Valencia CF


In [531]:
#Choose team to show information
#Fazer dropdown

team_matches=list()
next_matches=list()
next_home_away=list()
score_list=list()
goals_list=list()
home_away_list=list()
winner=list()
scheduled=1
for j in range (0, len(match_master_list)):
    for i in range (0, nr_games):
    
        if match_master_list[j][i]['homeTeam']['name']==team or match_master_list[j][i]['awayTeam']['name']==team:
            if match_master_list[j][i]['status']=='FINISHED':
                game= match_master_list[j][i]['homeTeam']['name']+" vs "+ match_master_list[j][i]['awayTeam']['name']
                if (team == game.split(" vs ")[0]):
                    home_away_list.append(1)
                    if(match_master_list[j][i]['score']['winner']=='HOME_TEAM'):
                        winner.append(1)
                    else:
                        winner.append(0)
                else:
                    home_away_list.append(0)
                    if(match_master_list[j][i]['score']['winner']=='AWAY_TEAM'):
                        winner.append(1)
                    else:
                        winner.append(0)
                        
                team_matches.append(game)
                result= str(match_master_list[j][i]['score']['fullTime']['homeTeam'])+ " - " + str(match_master_list[j][i]['score']['fullTime']['awayTeam'])
                score_list.append(result)
                goals_list.append(int(match_master_list[j][i]['score']['fullTime']['homeTeam'])+int(match_master_list[j][i]['score']['fullTime']['awayTeam']))
                #if ['score']['winner']
                #print(result)
                played=len(team_matches)
            if match_master_list[j][i]['status']!='FINISHED':
                
                game= match_master_list[j][i]['homeTeam']['name']+" vs "+ match_master_list[j][i]['awayTeam']['name']
                if (team == game.split(" vs ")[0]):
                    home_away_list.append(1)
                else:
                    home_away_list.append(0)
                print(game)
                team_matches.append(game)
                score_list.append(0)
                goals_list.append(0)
                winner.append(0)

Valencia CF vs CD Leganés
Valencia CF vs Getafe CF
Athletic Club vs Valencia CF
Valencia CF vs Deportivo Alavés
Club Atlético de Madrid vs Valencia CF
CA Osasuna vs Valencia CF
Valencia CF vs Sevilla FC
RCD Espanyol de Barcelona vs Valencia CF
Valencia CF vs Granada CF
Real Betis Balompié vs Valencia CF
Valencia CF vs Villarreal CF
Levante UD vs Valencia CF
Valencia CF vs Real Madrid CF
Real Valladolid CF vs Valencia CF
Valencia CF vs SD Eibar
RCD Mallorca vs Valencia CF
Valencia CF vs FC Barcelona
Valencia CF vs RC Celta de Vigo
Getafe CF vs Valencia CF
Valencia CF vs Club Atlético de Madrid
Valencia CF vs Real Betis Balompié
Deportivo Alavés vs Valencia CF
Valencia CF vs Levante UD
Real Madrid CF vs Valencia CF
Real Sociedad de Fútbol vs Valencia CF
Valencia CF vs CA Osasuna
SD Eibar vs Valencia CF
Villarreal CF vs Valencia CF
Valencia CF vs Athletic Club
Granada CF vs Valencia CF
Valencia CF vs Real Valladolid CF
CD Leganés vs Valencia CF
Valencia CF vs RCD Espanyol de Barcelona
Sev

In [532]:
#Calling DataFrame constructor after zipping 
# both lists, with columns specified 
df = pd.DataFrame(list(zip(team_matches, home_away_list, goals_list, winner)), 
               columns =['Match', 'Home/Away', 'Goals', 'Won']) 

In [533]:
labels, uniques = pd.factorize(df.Match)
df['Match']=labels

In [539]:
X=df.filter(['Match', 'Home/Away'], axis=1)[:played]
y=df['Goals']
y=y[:played]
next_g=df.filter(['Match', 'Home/Away'], axis=1)[played:]
X=np.matrix(X)
y=np.array(y)
next_g=np.matrix(next_g)

div=int(len(X)*0.75)
print(div)
ngoals=2.5
mode='Goals'

if(mode=='Goals'):
    for i in range (0, len(y)):
        if y[i]>ngoals:
            y[i]=1
        else:
            y[i]=0

X_train=X[:div]
y_train=y[:div]
X_test=X[div:]
y_test=y[div:]

60


In [540]:
#XGBoost
#Ex https://www.kdnuggets.com/2017/03/simple-xgboost-tutorial-iris-dataset.html
import xgboost as xgb
from sklearn.model_selection import GridSearchCV


dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
next_g_matrix=xgb.DMatrix(next_g[0])

param = {
    'max_depth': 3,  # the maximum depth of each tree
    'eta': 0.01,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'binary:logistic',  # error evaluation for multiclass training
    'eval_metric' : 'logloss'
}
num_round = 5000 # the number of training iterations

bst = xgb.train(param, dtrain, num_round)


preds = bst.predict(dtest)
print(preds)
import numpy as np
for i in range (0, len(preds)):
    if preds[i]>=0.5:
        preds[i]=1
    else:
        preds[i]=0
print ("Accuracy:", accuracy_score(y_test, preds))

[0.95372105 0.21696138 0.0797769  0.8106906  0.2352768  0.9768314
 0.02217005 0.9356937  0.23863901 0.95372105 0.89650846 0.22799566
 0.15513773 0.23863901 0.709543   0.23863901 0.95372105 0.50527865
 0.67365986 0.8647455 ]
Accuracy: 0.55


In [541]:
print('O próximo jogo tem uma probabilidade de: ', bst.predict(next_g_matrix)[0]*100, '% para haver mais que', ngoals, 'golos')

O próximo jogo tem uma probabilidade de:  49.90643560886383 % para haver mais que 2.5 golos


In [542]:
preds

array([1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1.,
       1., 1., 1.], dtype=float32)

In [543]:
y_test

array([0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1])